In [135]:
import gensim
import numpy
import logging
import keras
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
#from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import KFold
#from sklearn.preprocessing import LabelEncoder
#from sklearn.pipeline import Pipeline
from itertools import zip_longest, count
import json 
import nltk
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Merge, Dropout, Reshape, RepeatVector, Masking
from keras.layers.recurrent import LSTM,GRU
from collections import Counter
from keras.layers.wrappers import TimeDistributed

In [136]:
# Load Files into Memory.

image_features = numpy.load('smaller_merged_train.npy')
with open ('smaller_merged_train.json') as f:
    image_captions = json.load(f)

# Remove this to end fixed testing with 5 data features
image_features = image_features[:5]

In [137]:
# Generates a list containing all sentences appended with start and termination symbols.
# select_list contains the selector indexed sentence.

input_list = []
select_list = []
selector = 0
for i in range(len(image_captions)):
    sent_temp = []
    for j in range(len(image_captions[i][1])):
        temp = ['<s>']
        temp.extend(image_captions[i][1][j])
        temp.append("</s>")        
        input_list.append(temp)
        if (j==selector):
            select_list.append(temp)

# Remove this to end fixed testing with 5 sentences
select_list = select_list[:5]

In [138]:
# Generation of Dictionary and vocab_size parameter

word_dict = []

for i in range(len(select_list)):
    for j in range(len(select_list[i])):
        if (select_list[i][j] not in word_dict):
            word_dict.append(select_list[i][j])

# Vocab size increased by 1 to avoid 0 as an integer label.

vocab_size = len(word_dict) + 1

print("Size of this Vocabulary is %r" %vocab_size) 

#Generation of MAXLEN Parameter            

MAXLEN = 0

for i in range(len(select_list)):
    if (len(select_list[i])>MAXLEN):
        MAXLEN = len(select_list[i])

print("Maxlen for this Data is %r "%MAXLEN)            


Size of this Vocabulary is 46
Maxlen for this Data is 18 


In [139]:
char_labels = {ch:i for i, ch in enumerate(word_dict)}

labels_char = {i:ch for i, ch in enumerate(word_dict)}

In [144]:
# New Awesomeness here!

x_train = []
y_train = []
image_train = []



for i in range(len(select_list)):
    for j in range(len(select_list[i])):
        if (j>0):
            x_train.append(select_list[i][:j])
            y_train.append(select_list[i][j])
            image_train.append(image_features[i])

print (x_train)
x_train = numpy.asarray(x_train)
image_train = numpy.asarray(image_train)


# Generation of nb_samples parameter 

nb_samples = len(x_train)

#Convert Streams to their Integer valued counterparts



for i in range(len(x_train)):
    for j in range(len(x_train[i])):
        x_train[i][j] = char_labels[x_train[i][j]] + 1
        
x_train = numpy.asarray(x_train)


x_train = sequence.pad_sequences(x_train,maxlen = MAXLEN, padding = 'post')

for i in range(len(y_train)):
        y_train[i] = char_labels[y_train[i]] + 1

# y_train = sequence.pad_sequences(y_train,maxlen = MAXLEN, padding = 'post')


y_out = numpy.zeros((nb_samples,vocab_size))

for i in range(len(y_train)):
    y_out[i][y_train[i]] = 1

y_train = y_out

print (image_train.shape)
print (x_train.shape)
print(y_train.shape)

[['<s>'], ['<s>', 'Two'], ['<s>', 'Two', 'people'], ['<s>', 'Two', 'people', 'in'], ['<s>', 'Two', 'people', 'in', 'dim'], ['<s>', 'Two', 'people', 'in', 'dim', 'lit'], ['<s>', 'Two', 'people', 'in', 'dim', 'lit', 'room'], ['<s>', 'Two', 'people', 'in', 'dim', 'lit', 'room', 'playing'], ['<s>', 'Two', 'people', 'in', 'dim', 'lit', 'room', 'playing', 'Nintendo'], ['<s>'], ['<s>', 'A'], ['<s>', 'A', 'white'], ['<s>', 'A', 'white', 'sink'], ['<s>', 'A', 'white', 'sink', 'sitting'], ['<s>', 'A', 'white', 'sink', 'sitting', 'next'], ['<s>', 'A', 'white', 'sink', 'sitting', 'next', 'to'], ['<s>', 'A', 'white', 'sink', 'sitting', 'next', 'to', 'a'], ['<s>', 'A', 'white', 'sink', 'sitting', 'next', 'to', 'a', 'toilet'], ['<s>', 'A', 'white', 'sink', 'sitting', 'next', 'to', 'a', 'toilet', 'in'], ['<s>', 'A', 'white', 'sink', 'sitting', 'next', 'to', 'a', 'toilet', 'in', 'a'], ['<s>', 'A', 'white', 'sink', 'sitting', 'next', 'to', 'a', 'toilet', 'in', 'a', 'bathroom'], ['<s>', 'A', 'white', 'si

In [80]:





# Shite below this line!!!!----------------------------------------------------------------------------------

# # Generate Input and Output Streams

# x_train = []
# y_train = []

# for i in range(len(select_list)):
#     x_train.append(select_list[i][:-1])
#     y_train.append(select_list[i][1:])

# x_train = numpy.asarray(x_train)


# #Convert Streams to their Integer valued counterparts

# for i in range(len(x_train)):
#     for j in range(len(x_train[i])):
#         x_train[i][j] = char_labels[x_train[i][j]] + 1
        
# x_train = numpy.asarray(x_train)


# x_train = sequence.pad_sequences(x_train,maxlen = MAXLEN, padding = 'post')

        
# for i in range(len(y_train)):
#     for j in range(len(y_train[i])):
#         y_train[i][j] = char_labels[y_train[i][j]]
        
# y_train = sequence.pad_sequences(y_train,maxlen = MAXLEN, padding = 'post')

# # Convert Outputs to One-Hot form

# y_out = numpy.zeros((nb_samples,MAXLEN,vocab_size))

# for i in range(len(y_train)):
#     for j in range(len(y_train[i])):
#         y_out[i][j][y_train[i][j]] = 1

# y_train = y_out



In [193]:
num_hidden_units_mlp = 1024
image_model = Sequential()
image_model.add(Dense(128,input_dim = 4096))

language_model = Sequential()
language_model.add(Embedding(vocab_size, 256, input_length=MAXLEN))
language_model.add(GRU(output_dim=128, return_sequences=True))
language_model.add(TimeDistributed(Dense(128)))

image_model.add(RepeatVector(MAXLEN))

model = Sequential()
model.add(Merge([image_model, language_model], mode='concat', concat_axis=-1))

model.add(GRU(256, return_sequences=False))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [194]:
# Keep best weights and save callback history

# define the checkpoint

filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [195]:
model.fit([image_train, x_train], y_train, batch_size=1, nb_epoch=40)

Epoch 1/40
61/61 [==============================] - 4s - loss: 4.6338     
Epoch 2/40
61/61 [==============================] - 3s - loss: 3.9537     
Epoch 3/40
61/61 [==============================] - 3s - loss: 3.7213     
Epoch 4/40
61/61 [==============================] - 3s - loss: 3.6365     
Epoch 5/40
61/61 [==============================] - 3s - loss: 3.4667     
Epoch 6/40
61/61 [==============================] - 3s - loss: 3.4591     
Epoch 7/40
61/61 [==============================] - 3s - loss: 3.3533     
Epoch 8/40
61/61 [==============================] - 3s - loss: 3.2340     
Epoch 9/40
61/61 [==============================] - 3s - loss: 3.2033     
Epoch 10/40
61/61 [==============================] - 3s - loss: 3.0931     
Epoch 11/40
61/61 [==============================] - 4s - loss: 3.0738     
Epoch 12/40
61/61 [==============================] - 4s - loss: 3.0728     
Epoch 13/40
61/61 [==============================] - 4s - loss: 3.0102     
Epoch 14/40
61/61 [==

In [196]:
x_test = x_train[9:21]
image_test = image_train[9:21]


# x_test = x_train[:7]
# images_test = image_train[:7]

# print (x_test)
# print (images_test)
prediction = model.predict_classes([images_test,x_test])
print (prediction)


7/7 [==============================] - 0s
[4 4 4 4 4 4 4]
